In [1]:
import numpy as np
import pandas as pd
from arrow import Arrow
from arrow import now

def make_date(arg, year, month, day):
    if np.isnan(arg[year]):
        return now(tz=None).date()
    return Arrow(year=int(arg[year]), month=int(arg[month]), day=int(arg[day]), ).date()

PRESIDENTS = '/kaggle/input/historical-us-president-physical-data-more/Historical Presidents Physical Data (More).csv'
df = pd.read_csv(filepath_or_buffer=PRESIDENTS, )
df['birth'] = df.apply(func=make_date, axis=1, args=('birth_year', 'birth_month', 'birth_day',))
df['death'] = df.apply(func=make_date, axis=1, args=('death_year', 'death_month', 'death_day',))

df.head()

,order,name,height_cm,height_in,weight_kg,weight_lb,body_mass_index,body_mass_index_range,birth_day,birth_month,...,term_end_day,term_end_month,term_end_year,term_end_date,presidency_begin_age,presidency_end_age,political_party,corrected_iq,birth,death
0,1,George Washington,188,74.0,79.4,175,22.5,Normal,22,2,...,4.0,3.0,1797.0,04-03-1797,57,65.0,Unaffiliated,140.0,1732-02-22,1799-12-14
1,2,John Adams,170,67.0,83.9,185,29.0,Overweight,30,10,...,4.0,3.0,1801.0,04-03-1801,61,65.0,Federalist,155.0,1735-10-30,1826-07-04
2,3,Thomas Jefferson,189,74.5,82.1,181,23.0,Normal,13,4,...,4.0,3.0,1809.0,04-03-1809,57,65.0,Democratic-Republican,160.0,1743-04-13,1826-07-04
3,4,James Madison,163,64.0,55.3,122,20.8,Normal,16,3,...,4.0,3.0,1817.0,04-03-1817,57,65.0,Democratic-Republican,160.0,1751-03-16,1836-06-28
4,5,James Monroe,183,72.0,85.7,189,25.6,Overweight,28,4,...,4.0,3.0,1825.0,04-03-1825,58,66.0,Democratic-Republican,139.0,1758-04-28,1831-07-04


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   order                  45 non-null     object 
 1   name                   45 non-null     object 
 2   height_cm              45 non-null     int64  
 3   height_in              45 non-null     float64
 4   weight_kg              45 non-null     float64
 5   weight_lb              45 non-null     int64  
 6   body_mass_index        45 non-null     float64
 7   body_mass_index_range  45 non-null     object 
 8   birth_day              45 non-null     int64  
 9   birth_month            45 non-null     int64  
 10  birth_year             45 non-null     int64  
 11  birth_date             45 non-null     object 
 12  birthplace             45 non-null     object 
 13  birth_state            45 non-null     object 
 14  death_day              39 non-null     float64
 15  death_mo

In [3]:
from plotly.express import timeline
timeline(data_frame=df, x_start='birth', x_end='death', y='name', height=900)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   order                  45 non-null     object 
 1   name                   45 non-null     object 
 2   height_cm              45 non-null     int64  
 3   height_in              45 non-null     float64
 4   weight_kg              45 non-null     float64
 5   weight_lb              45 non-null     int64  
 6   body_mass_index        45 non-null     float64
 7   body_mass_index_range  45 non-null     object 
 8   birth_day              45 non-null     int64  
 9   birth_month            45 non-null     int64  
 10  birth_year             45 non-null     int64  
 11  birth_date             45 non-null     object 
 12  birthplace             45 non-null     object 
 13  birth_state            45 non-null     object 
 14  death_day              39 non-null     float64
 15  death_mo